# VoxelNet

- Useful utilities: 2D IoU with oriented bounding boxes
- 3D IoU with oriented bounding boxes along z-axis
- Voxelization with point feature augmentation: VFE layer

In [11]:
import random

import torch
import torch.nn as nn
import torch_scatter
import numpy as np

import MinkowskiEngine as ME

## Useful utilities

In [15]:
# Reference: https://github.com/jw9730/ori-giou/blob/master/util/box_ops.py

class Lines:
    # N instances of (ax + by + c = 0)
    def __init__(self, v1, v2):  # v1, v2: [N, 2]
        self.a = v2[:, 1] - v1[:, 1]  # [N,]
        self.b = v1[:, 0] - v2[:, 0]
        self.c = v2[:, 0] * v1[:, 1] - v2[:, 1] * v1[:, 0]
        self.eps = 1e-10

    def __call__(self, p):  # [N, 2]
        return self.a * p[:, 0] + self.b * p[:, 1] + self.c  # [N,]

    def intersection(self, others):
        if not isinstance(others, Lines):
            return NotImplementedError
        w = self.a * others.b - self.b * others.a + self.eps
        inter = torch.stack([
            (self.b * others.c - self.c * others.b) / w,
            (self.c * others.a - self.a * others.c) / w], dim=-1)
        return inter  # [N, 2]


def box_center_to_corners(b):
    """
    Converts a set of oriented bounding boxes from
    centered representation (x_c, y_c, w, h, theta) to corner representation (x0, y0, ..., x3, y3).
    Arguments:
        b (Tensor[N, 6]): boxes to be converted. They are
            expected to be in (x_c, y_c, w, h, c, s) format.
            * c, s: unnormalized cos, sin
    Returns:
        c (Tensor[N, 8]): converted boxes in (x0, y0, ..., x3, y3) format, where
            the corners are sorted counterclockwise.
    """
    x_c, y_c, w, h, c, s = b.unbind(-1)  # [N,]
    center = torch.stack([x_c, y_c], dim=-1).repeat(1, 4)  # [N, 8]

    dx = 0.5 * w
    dy = 0.5 * h
    c = c + 1e-5
    s = s + 1e-5
    cos = c / ((c ** 2 + s ** 2).sqrt() + 1e-10)
    sin = s / ((c ** 2 + s ** 2).sqrt() + 1e-10)

    dxcos = dx * cos
    dxsin = dx * sin
    dycos = dy * cos
    dysin = dy * sin

    dxy = [- dxcos + dysin, - dxsin - dycos,
             dxcos + dysin,   dxsin - dycos,
             dxcos - dysin,   dxsin + dycos,
           - dxcos - dysin, - dxsin + dycos]

    return center + torch.stack(dxy, dim=-1)  # [N, 8]


def box_corners_to_center(corners):
    """
    Arguments:
        corners (Tensor[N, 8]): boxes to be converted. They are
            expected to be in (x0, y0, ..., x3, y3) format, where the corners are sorted counterclockwise.
    Returns:
        b (Tensor[N, 6]): converted boxes in centered
            (x_c, y_c, w, h, c, s) format.
            * c, s: sin, cos before sigmoid
    """

    x0,y0, x1,y1, x2,y2, x3,y3 = corners.unbind(-1)

    x_c = (x0 + x2) / 2
    y_c = (y0 + y2) / 2

    wsin, wcos, hsin, hcos = (y1 - y0,
                              x1 - x0,
                              x0 + x1,
                              y2 + y3)
    theta = torch.atan2(wsin, wcos)
    c = torch.cos(theta)
    s = torch.sin(theta)

    b = [x_c, y_c,
         (wsin ** 2 + wcos ** 2).sqrt(),
         (hsin ** 2 + hcos ** 2).sqrt(),
         c, s]
    return torch.stack(b, dim=-1)
    
    
def box_area(boxes):
    x0,y0, x1,y1, x2,y2, x3,y3 = boxes.unbind(-1)  # [N,]
    return ((x1 - x0) ** 2 + (y1 - y0) ** 2) ** 0.5 *\
           ((x3 - x0) ** 2 + (y3 - y0) ** 2) ** 0.5  # [N,]


def cuts(polygons, sizes, p, q):
    """
    vectorized polygon cut
    Arguments:
        polygons (Tensor[N, K, 2])
        sizes (Tensor[N,])
        p (Tensor[N, 2])
        q (Tensor[N, 2])
    Returns:
        new_polygons (Tensor[N, K+1, 2])
        new_sizes (Tensor[N,])
    new_polygons = list()
    """
    N = polygons.shape[0]
    K = polygons.shape[1]

    # Any point p with line(p) <= 0 is on the "inside" (or on the boundary),
    # any point p with line(p) > 0 is on the "outside".
    lines = Lines(p, q)

    polygons_abc = polygons  # [N, K, 2]
    polygons_bca = polygons.clone()
    polygons_bca[:, :-1, :] = polygons_abc[:, 1:, :]
    polygons_bca[torch.arange(N), sizes - 1, :] = polygons_abc[:, 0, :]

    v_abc = torch.stack([lines(polygons_abc[:, k, :]) for k in range(K)], dim=1)  # [N, K]
    v_bca = torch.stack([lines(polygons_bca[:, k, :]) for k in range(K)], dim=1)  # [N, K]

    # use new_polygons as stack and new_sizes as stack pointer
    # iterate and push new points
    new_polygons = torch.zeros(N, K + 1, 2).to(polygons.device).fill_(1e5)  # [N, K + 1, 2]
    new_sizes = torch.zeros(N).to(polygons.device).long()  # [N,]
    for k in range(K):
        s = polygons_abc[:, k, :].clone()  # [N, 2]
        t = polygons_bca[:, k, :].clone()
        s_v = v_abc[:, k].clone()  # [N,]
        t_v = v_bca[:, k].clone()

        # only keep valid points
        valid = sizes > k  # [N,]
        s[~valid, :] = 0
        t[~valid, :] = 0
        s_v[~valid] = 0
        t_v[~valid] = 0

        # push preserved points to stack
        mask = (s_v <= 0) & valid
        push = s.clone()  # [N, 2]
        keep = new_polygons[torch.arange(N), new_sizes - 1, :]  # [N, 2]

        push[~mask, :] = 0
        keep[mask, :] = 0

        new_sizes = new_sizes + mask.long().squeeze(-1)
        new_polygons[torch.arange(N), new_sizes - 1, :] = push + keep

        # push intersection points to stack
        mask = (s_v * t_v < 0) & valid
        push = lines.intersection(Lines(s, t))  # [N, 2]
        keep = new_polygons[torch.arange(N), new_sizes - 1, :]  # [N, 2]

        push[~mask, :] = 0
        keep[mask, :] = 0

        new_sizes = new_sizes + mask.long().squeeze(-1)
        new_polygons[torch.arange(N), new_sizes - 1, :] = push + keep

    return new_polygons, new_sizes


def box_inter_tensor(boxes1, boxes2):
    """
    Finds intersection convex polygon using sequential cut, then computes its area by counterclockwise cross product.
    https://stackoverflow.com/questions/44797713/calculate-the-area-of-intersection-of-two-rotated-rectangles-in-python/45268241
       Arguments:
           boxes1, boxes2 (Tensor[N, 8], Tensor[M, 8]): boxes to compute area of intersection. They are
               expected to be in (x0, y0, ..., x3, y3) format, where the corners are sorted counterclockwise.
       Returns:
           inter (Tensor[N, M]) pairwise matrix, where N = len(boxes1) and M = len(boxes2)
    """
    N = boxes1.shape[-2]
    M = boxes2.shape[-2]

    boxes1 = boxes1.reshape([-1, 4, 2])
    boxes2 = boxes2.reshape([-1, 4, 2])

    # vectorized intersection computation
    inter_xy = boxes1.unsqueeze(1).expand(-1, M, -1, -1).reshape([N * M, 4, 2])  # [N * M, 4, 2]
    cut_rect = boxes2.unsqueeze(0).expand(N, -1, -1, -1).reshape([N * M, 4, 2])  # [N * M, 4, 2]
    sizes = torch.zeros([N * M,]).to(boxes1.device).long().fill_(4)  # [N, M]

    inter_xy, sizes = cuts(inter_xy, sizes, cut_rect[:, 0, :], cut_rect[:, 1, :])  # [N * M, 5, 2]
    inter_xy, sizes = cuts(inter_xy, sizes, cut_rect[:, 1, :], cut_rect[:, 2, :])  # [N * M, 6, 2]
    inter_xy, sizes = cuts(inter_xy, sizes, cut_rect[:, 2, :], cut_rect[:, 3, :])  # [N * M, 7, 2]
    inter_xy, sizes = cuts(inter_xy, sizes, cut_rect[:, 3, :], cut_rect[:, 0, :])  # [N * M, 8, 2]

    # compute area
    inter_abc = inter_xy  # [N*M, 8, 2]
    inter_bca = inter_abc.clone()
    inter_bca[:, :-1, :] = inter_abc[:, 1:, :]
    inter_bca[torch.arange(N * M), sizes - 1, :] = inter_abc[:, 0, :]

    inter = inter_abc[:, :, 0] * inter_bca[:, :, 1] - \
            inter_abc[:, :, 1] * inter_bca[:, :, 0]

    sizes = sizes.unsqueeze(-1).expand([-1, 8])  # [N * M, 8]
    inter[sizes <= 2] = 0
    inter[sizes <= torch.arange(8).unsqueeze(0).to(boxes1.device)] = 0

    return 0.5 * inter.reshape([N, M, -1]).sum(dim=-1)  # [N, M]


def box_iou(boxes1, boxes2, return_areas=False):
    """
    Arguments:
        boxes1, boxes2 (Tensor[N, 8], Tensor[M, 8]): boxes to compute IoU. They are
            expected to be in (x0, y0, ..., x3, y3) format, where the corners are sorted counterclockwise.
    Returns:
        iou: [N, M] pairwise matrix, where N = len(boxes1) and M = len(boxes2)
        union: [N, M] pairwise matrix
    """
    area1 = box_area(boxes1)  # [N,]
    area2 = box_area(boxes2)  # [M,]

    """
    inter = torch.zeros([boxes1.shape[-2], boxes2.shape[-2]]).to(boxes1.device)  # [N, M]
    for n in range(boxes1.shape[-2]):
        for m in range(boxes2.shape[-2]):
            inter[n, m] = box_inter(boxes1[n, :], boxes2[m, :])
    """
    inter = box_inter_tensor(boxes1, boxes2)

    union = area1.unsqueeze(-1) + area2.unsqueeze(-2) - inter  # [N, M]

    iou = inter / union  # [N, M]
    
    if return_areas:
        return iou, union, area1, area2
    else:
        return iou, union


def generate_random_boxes(n):
    boxes = torch.rand(n, 6)
    boxes[:, 4:] = torch.randn(n, 2)
    
    return boxes

In [16]:
N = 100
M = 50


boxes1 = generate_random_boxes(N) # x_c, y_c, w, h, c, s
boxes2 = generate_random_boxes(M)

iou, union = box_iou(box_center_to_corners(boxes1), box_center_to_corners(boxes2))

print(iou.shape)
print(union.shape)
print(iou)

torch.Size([100, 50])
torch.Size([100, 50])
tensor([[0.0462, 0.0491, 0.0420,  ..., 0.0399, 0.1563, 0.0023],
        [0.0000, 0.0000, 0.0000,  ..., 0.0921, 0.0000, 0.0000],
        [0.1530, 0.0006, 0.0910,  ..., 0.0000, 0.0023, 0.1765],
        ...,
        [0.0596, 0.0000, 0.0000,  ..., 0.0000, 0.1412, 0.0000],
        [0.0000, 0.0000, 0.0549,  ..., 0.1709, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0315, 0.0000]])


## 3D IoU with oriented boxes (x, y, z, l, w, h, yaw)
- Input: a set of 3D oriented boxes (N, 7), the other set of 3D oriented boxes (M, 7)
- Output: IoU and Union, (N, M)

In [46]:
def box_iou_3d(boxes1, boxes2):
    N = len(boxes1)
    M = len(boxes2)
    
    # 1. Calculate 2D IoU.
    def convert_3d_to_2d(boxes):
        return torch.cat([
            boxes[:, :2], boxes[:, 3:5], torch.cos(boxes[:, -1])[..., None], torch.sin(boxes[:, -1])[..., None]
        ], dim=1)
    
    boxes1_2d = convert_3d_to_2d(boxes1)
    boxes2_2d = convert_3d_to_2d(boxes2)
    
    iou, union, area1, area2 = box_iou(
        box_center_to_corners(boxes1_2d), 
        box_center_to_corners(boxes2_2d),
        return_areas=True
    )
    
    # 2. Find z-axis intersection.
    boxes1_dz = 0.5 * boxes1[:, 5]
    boxes1_z_max = boxes1[:, 2] + boxes1_dz
    boxes1_z_min = boxes1[:, 2] - boxes1_dz
    boxes2_dz = 0.5 * boxes2[:, 5]
    boxes2_z_max = boxes2[:, 2] + boxes2_dz
    boxes2_z_min = boxes2[:, 2] - boxes2_dz
    inter_z_max = torch.min(torch.cat([
        boxes1_z_max[..., None].repeat(1, M).unsqueeze(-1),
        boxes2_z_max[None, ...].repeat(N, 1).unsqueeze(-1)
    ], dim=-1), dim=-1)[0]
    inter_z_min = torch.max(torch.cat([
        boxes1_z_min[..., None].repeat(1, M).unsqueeze(-1),
        boxes2_z_min[None, ...].repeat(N, 1).unsqueeze(-1)
    ], dim=-1), dim=-1)[0]
    inter_dz = inter_z_max - inter_z_min # (N, M)
    inter_dz = inter_dz.clamp(min=0)
    
    # 2. Cacluate 3D IoU.
    iou_3d = torch.zeros(N, M) # init
    
    inter = iou * union
    inter_vol = inter * inter_dz # (N, M)
    vol1 = area1 * boxes1[:, 5]
    vol2 = area2 * boxes2[:, 5]
    
    iou_3d = inter_vol / (vol1.unsqueeze(-1) + vol2.unsqueeze(-2) - inter_vol)
    
    return iou_3d

In [47]:
N = 100
M = 50

boxes1 = torch.rand(N, 7) # (x, y, z, l, w, h, yaw)
boxes2 = torch.rand(M, 7)

iou = box_iou_3d(boxes1, boxes2)

print(iou.shape)
print(iou)
print(iou.max())
print(iou.min())

torch.Size([100, 50])
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 6.4489e-02, 6.5585e-03,
         0.0000e+00],
        [6.0891e-05, 0.0000e+00, 0.0000e+00,  ..., 2.1790e-02, 0.0000e+00,
         6.2031e-02],
        [1.3805e-01, 0.0000e+00, 0.0000e+00,  ..., 4.5280e-02, 0.0000e+00,
         8.0448e-02],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 5.3248e-03, 0.0000e+00,
         1.3830e-03],
        [5.3294e-02, 0.0000e+00, 1.3963e-02,  ..., 1.6068e-01, 0.0000e+00,
         2.8434e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         2.1375e-02]])
tensor(0.5059)
tensor(0.)


## Recap of Voxelization

In [50]:
def ravel_hash(voxel_indices):
    # 1. Find the maximum value of each axis.
    max_index = np.max(voxel_indices, axis=0).astype(np.uint64) + 1
    
    # 2. Hashing
    keys = np.zeros(len(voxel_indices), dtype=np.uint64)
    for d in range(voxel_indices.shape[1] - 1): # dimension
        keys += voxel_indices[:, d]
        keys *= max_index[d + 1]
    keys += voxel_indices[:, -1]
    
    return keys


def voxelize(points, voxel_size):
    # 1. Make all the coordinates positive
    origin = np.min(points, axis=0)
    points = points - origin
    
    # 2. Make the voxel indices and hash keys
    voxel_indices = np.floor(points / voxel_size).astype(np.uint64)
    keys = ravel_hash(voxel_indices)
    
    # 3. Find the unique voxel indices and the mappings.
    _, unique_mapping, inverse_mapping = np.unique(keys, return_index=True, return_inverse=True)
    unique_voxel_indices = voxel_indices[unique_mapping]
    
    return origin, unique_voxel_indices, unique_mapping, inverse_mapping

#### The (pooled) voxel features can be calculated from point-wise features using `torch_scatter`!

In [51]:
N = 100
C = 4 # the dimension of point-wise features
l = 1. # voxel size

points = torch.randn(N, 3)
features = torch.randn(N, C)

# First, voxelize the points.
origin, voxels, unique_map, inverse_map = voxelize(points.numpy(), l)
M = len(voxels)
print(f"[voxelization] {N} points -> {M} voxels")

# Then, calculate voxel features.
# Option 1: MaxPool
voxel_features, _ = torch_scatter.scatter_max(features, torch.from_numpy(inverse_map), dim=0, dim_size=M)
print(voxel_features.shape)

# Option 2: AvgPool
voxel_features = torch_scatter.scatter_mean(features, torch.from_numpy(inverse_map), dim=0, dim_size=M)
print(voxel_features.shape)

[voxelization] 100 points -> 49 voxels
torch.Size([49, 4])
torch.Size([49, 4])


## Voxel Feature Extraction(VFE) Layer
- Input: points (N, 3), point features (N, C_in), voxel length
- Output: point-wise output features (N, C_out)

In [54]:
class VFELayer(nn.Module):
    
    def __init__(self, in_channels, out_channels, voxel_length):
        super(VFELayer, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.l = voxel_length
        
        # Feed-forward network
        self.ffn = nn.Sequential(
            nn.Linear(3 + in_channels + 3, out_channels, bias=False),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, points, point_features):
        # 1. Voxelization
        _, _, unique_map, inverse_map = voxelize(points.numpy(), self.l)
        M = len(unique_map)
        inverse_map = torch.from_numpy(inverse_map)
        
        # 2. Calculate the centroid of each voxel.
        centroids = torch_scatter.scatter_mean(points, inverse_map, dim=0, dim_size=M)
        rel_pos = points - centroids[inverse_map]
        aug_point_features = torch.cat([points, point_features, rel_pos], dim=-1)
        
        # 3. Feed-forward the augmented point features.
        aug_point_features = self.ffn(aug_point_features)
        
        # 4. Locally max-pool the features to calculate voxel features.
        voxel_features, _ = torch_scatter.scatter_max(aug_point_features, inverse_map, dim=0, dim_size=M)
        
        # 5. Point-wise concatenation.
        out_point_features = torch.cat([aug_point_features, voxel_features[inverse_map]], dim=-1)
        
        return out_point_features

In [56]:
N = 100
C_in = 1
C_out = 8
l = 1. # voxel length

points = torch.randn(N, 3)
features = torch.randn(N, C_in)

vfe_layer = VFELayer(C_in, C_out, l)
out_features = vfe_layer(points, features)

print(out_features.shape)
print(out_features)

torch.Size([100, 16])
tensor([[1.3251e-03, 1.7352e+00, 0.0000e+00,  ..., 0.0000e+00, 2.8377e+00,
         1.9405e+00],
        [0.0000e+00, 5.4683e-01, 0.0000e+00,  ..., 0.0000e+00, 2.3776e+00,
         2.0366e+00],
        [0.0000e+00, 0.0000e+00, 1.1855e+00,  ..., 1.7839e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [7.1086e-01, 0.0000e+00, 8.3289e-01,  ..., 9.8345e-01, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 5.0196e-01,
         1.5722e-01],
        [3.1039e-01, 8.0174e-01, 0.0000e+00,  ..., 0.0000e+00, 1.5648e+00,
         1.1002e+00]], grad_fn=<CatBackward0>)
